<div  align='center'><img src='https://s3.amazonaws.com/weclouddata/images/logos/wcd_logo_new_2.png' width='30%'></div >

<p style="font-size:20px;text-align:center"><b><font color='#F39A54'>Data Engineering Diploma</font></b></p>

<h2 align='center'> WeCloudData Data Engineer Spark Exercise 2 </h2>

<br>

Please download data from [HERE](s3://weclouddata/data/data/pyspark_exercises_data.zip)

# Section 4. Search and Filter DataFrames in PySpark Homework Solutions

In [0]:
# First let's create our PySpark instance
# import findspark
# findspark.init()

import pyspark # only run after findspark.init()
from pyspark.sql import SparkSession
# May take awhile locally
spark = SparkSession.builder.appName("FunctionsHW").getOrCreate()

cores = spark._jsc.sc().getExecutorMemoryStatus().keySet().size()
print("You are working with", cores, "core(s)")
spark

## Read in the DataFrame for this Notebook

We will be continuing to use the fifa19.csv file for this notebook. Make sure that you are writting the correct path to the file. 

In [0]:
fifa = spark.read.csv('Datasets/fifa19.csv',inferSchema=True,header=True)

Use the .toPandas() method to view the first few lines of the dataset so we know what we are working with. 

In [0]:
fifa.limit(4).toPandas()

Now print the schema of the dataset so we can see the data types of all the varaibles. 

In [0]:
print(fifa.printSchema())

In [0]:
from pyspark.sql.functions import *

## S4.1. Select the Name and Position of each player in the dataframe

In [0]:
fifa.select(['Name','Position','Release Clause']).show(5,False)

## S4.1.1 Display the same results from above sorted by the players names

In [0]:
fifa.select(['Name','Position']).orderBy('Name').show(5)

## S4.2. Select only the players who belong to a club beginning with FC

In [0]:
# One way
fifa.select("Name","Club").where(fifa.Club.like("FC%")).show(5, False)

In [0]:
# Another way 
fifa.select("Name","Club").where(fifa.Club.startswith("FC")).limit(4).toPandas()

## S4.3. Who is the oldest player in the dataset and how old are they?

Display only the name and age of the oldest player.

In [0]:
fifa.select("Name","Age").sort(desc("Age")).show(1)

## S4.4. Select only the following players from the dataframe:

 - L. Messi
 - Cristiano Ronaldo

In [0]:
fifa[fifa.Name.isin("L. Messi", "Cristiano Ronaldo")].limit(4).toPandas()

## S4.5. Can you select the first character from the Release Clause variable which indicates the currency used?

In [0]:
fifa.select("Release Clause",fifa["Release Clause"].substr(1,1)).show(5,False)

## S4.6. Can you select only the players who are over the age of 40?

In [0]:
fifa.filter("Age>40").limit(4).toPandas()

### That's is for now... Great Job!

# Section 5. Joining and Appending DataFrames in PySpark HW Solutions

Now it's time to test your knowledge and further engrain the concepts we touched on in the lectures. Let's go ahead and get started.




**As always let's start our Spark instance.**

In [0]:
# import findspark
# findspark.init()

import pyspark # only run after findspark.init()
from pyspark.sql import SparkSession
# May take awhile locally
spark = SparkSession.builder.appName("joins").getOrCreate()

cores = spark._jsc.sc().getExecutorMemoryStatus().keySet().size()
print("You are working with", cores, "core(s)")
spark

In [0]:
import os

path = "Datasets/uw-madison-courses/"

df_list = []
for filename in os.listdir(path):
    if filename.endswith(".csv"):
        filename_list = filename.split(".") #separate path from .csv
        df_name = filename_list[0]
        df = spark.read.csv(path+filename,inferSchema=True,header=True)
        df.name = df_name
        df_list.append(df_name)
        exec(df_name + ' = df')
        
# QA
print("Full list of dfs:")
print(df_list)

Now check the contents of a few of the dataframes that were read in above.

In [0]:
grade_distributions.limit(4).toPandas()

## S5.1.0. Can you assign the room numbers to each section of each course?

Show only the rooms uuid, facility code, room number, term code and the name of the course from the course_offerings table.

In [0]:
step1 = rooms.join(sections, rooms.uuid == sections.room_uuid, how='left').select([rooms.uuid,rooms.facility_code,sections.course_offering_uuid,'number'])
step1.limit(4).toPandas()

In [0]:
step2 = step1.join(course_offerings, step1.course_offering_uuid == course_offerings.uuid, how='left').select([rooms.uuid,rooms.facility_code,'number','term_code','name'])
step2.limit(4).toPandas()

## S5.1.1. Now show same output as above but for only facility number 0469 (facility_code)

In [0]:
step3 = step2.filter(step2.facility_code == "0469")
step3.limit(4).toPandas()

## S5.2. Count how many sections are offered for each subject for each facility

*Note: this will involve a groupby*

In [0]:
step1 = subjects.join(subject_memberships, subjects.code == subject_memberships.subject_code, how='inner').select(['name','course_offering_uuid']).withColumnRenamed('name','subject_name')
step1.limit(4).toPandas()

In [0]:
step2 = step1.join(sections, step1.course_offering_uuid == sections.course_offering_uuid, how='left').select(['subject_name','room_uuid'])
step2.limit(4).toPandas()

In [0]:
# I added a filter to make this a little simpler
step3 = step2.join(rooms, step2.room_uuid == rooms.uuid, how='left').filter('facility_code IN("0140","0545","0469","0031")').select(['subject_name','facility_code','room_code'])
step3.limit(4).toPandas()

In [0]:
# Option 1: Group by facility code and do a count
step3.groupBy('facility_code','subject_name').count().orderBy("facility_code").show(10, False) # False prevents truncation of column content

In [0]:
# Option 2: Groupby subject name and pivot the facility code
# to see each facility side by side within each subject
step3.groupBy("subject_name").pivot("facility_code").count().show(10, False)

## S5.3. What are the hardest classes?

Let's see if we can figure out which classes are the hardest by seeing how many students failed. Note that you will first need to aggregate the grades table by the course uuid to include all sections. Show the name of the course as well that you will need to get from the course_offering table.

In [0]:
grade_distributions.limit(4).toPandas()

In [0]:
course_offerings.limit(4).toPandas()

In [0]:
step1 = grade_distributions.groupBy("course_offering_uuid").sum("f_count")
step1.limit(4).toPandas()

In [0]:
step2 = step1.join(course_offerings, step1.course_offering_uuid == course_offerings.uuid, how='left').select(['name','sum(f_count)']).orderBy("sum(f_count)")
step2.toPandas().tail(5)

## Challenge Question: Automating data entry errors

We see in the dataframe below that there are several typos of various animal names. If this was a large database of several millions of records, correcting these errors would be way too labor intensive. How can we automate correcting these errors?

*Hint: Leven...*

In [0]:
values = [('Monkey',10),('Monkay',36),('Mnky',123), \
          ('Elephant',48),('Elefant',16),('Ellafant',1), \
          ('Hippopotamus',48),('Hipopotamus',16),('Hippo',1)]
zoo = spark.createDataFrame(values,['Animal','age'])
zoo.show()

In [0]:
# With the levenshtein distance!
from pyspark.sql.functions import *
from pyspark.sql.types import *

# First we create a dataframe with the 3 options we want to choose from
options = spark.createDataFrame(['Monkey', 'Elephant', 'Hippopotamus'], StringType())
options.show()

In [0]:
# And then we join the two dataframes together with a condition >5
results = zoo.join(options, levenshtein(zoo["Animal"], options["value"]) < 5, 'left')
results.show()

So we can see here that all of our values were correctly identified except for "Hippo" which was just way too different from "Hippopotamus" to get correctly identified. So this solution won't work for EVERY case, but we can see here that it did a great job correcting simple gramatical errors. 

# Section 6. Aggregating DataFrames in PySpark HW Solutions


In [0]:
# import findspark
# findspark.init()

import pyspark # only run after findspark.init()
from pyspark.sql import SparkSession
# May take awhile locally
spark = SparkSession.builder.appName("aggregate").getOrCreate()

cores = spark._jsc.sc().getExecutorMemoryStatus().keySet().size()
print("You are working with", cores, "core(s)")
spark

## Read in the dataFrame for this Notebook

In [0]:
airbnb = spark.read.csv('nyc_air_bnb.csv',inferSchema=True,header=True)

In [0]:
airbnb.limit(5).toPandas()

In [0]:
print(airbnb.printSchema())

Notice here that some of the columns that are obviously numeric have been incorrectly identified as "strings". Let's edit that. Otherwise we cannot aggregate any of the numeric columns.

In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

df = airbnb.withColumn("price", airbnb["price"].cast(IntegerType())) \
        .withColumn("minimum_nights", airbnb["minimum_nights"].cast(IntegerType())) \
        .withColumn("number_of_reviews", airbnb["number_of_reviews"].cast(IntegerType())) \
        .withColumn("reviews_per_month", airbnb["reviews_per_month"].cast(IntegerType())) \
        .withColumn("calculated_host_listings_count", airbnb["calculated_host_listings_count"].cast(IntegerType()))
#QA
print(df.printSchema())
df.limit(5).toPandas()

### Alright now we are ready to dig in!


## S6.1. How many rows are in this dataset?

In [0]:
df.count()

## S6.2. How many total reviews does each host have?

In [0]:
df.groupBy("host_id").sum('number_of_reviews').show(10)

## S6.3. Show the min and max of all the numeric variables in the dataset

In [0]:
limit_summary = df.select("price","minimum_nights","number_of_reviews","last_review","reviews_per_month","calculated_host_listings_count","availability_365").summary("min","max")
limit_summary.toPandas()

## S6.4. Which host had the highest number of reviews?

Only display the top result.

Bonus: format the column names

In [0]:
from pyspark.sql import functions
df.groupBy("host_id").agg(sum("number_of_reviews").alias("Reviews")).orderBy(sum("number_of_reviews").desc()).show(1) 

## S6.5. On average, how many nights did most hosts specify for a minimum?

In [0]:
df.agg({'minimum_nights':'avg'}).withColumnRenamed("avg(minimum_nights)", "Avg Min Nights").show()

In [0]:
df.agg(mean(df.minimum_nights)).show()

## S6.6. What is the most expensive neighbourhood to stay in on average?

Note: only show the one result

In [0]:
result = df.groupBy("neighbourhood").agg(avg(df.price).alias('avg_price'))
result.orderBy(result.avg_price.desc()).show(1) 

## S6.7. Display a two by two table that shows the average prices by room type (private and shared only) and neighborhood group (Manhattan and Brooklyn only)

In [0]:
df.filter("room_type IN('Private room','Shared room')").groupBy("room_type").pivot("neighbourhood_group", ["Manhattan", "Brooklyn"]).avg('price').show(100)

# Section 7. SQL Options in Spark HW Solutions

Alirght let's apply what we learned in the lecture to a new dataset!

**But first!**

Let's start with Spark SQL. But first we need to create a Spark Session!

In [0]:
# import findspark
# findspark.init()

import pyspark # only run after findspark.init()
from pyspark.sql import SparkSession
# May take awhile locally
spark = SparkSession.builder.appName("SparkSQLHWSolutions").getOrCreate()

cores = spark._jsc.sc().getExecutorMemoryStatus().keySet().size()
print("You are working with", cores, "core(s)")
spark

## Read in our DataFrame for this Notebook

For this notebook we will be using the Google Play Store csv file attached to this lecture. Let's go ahead and read it in. 

### About this dataset

Contains a list of Google Play Store Apps and info about the apps like the category, rating, reviews, size, etc. 

**Source:** https://www.kaggle.com/lava18/google-play-store-apps

In [0]:
path = 'Datasets/'

googlep = spark.read.csv(path+"googleplaystore.csv",header=True,inferSchema=True)

## First things first

Let's check out the first few lines of the dataframe to see what we are working with

In [0]:
# This is way better
googlep.limit(5).toPandas()

As well as the schema to make sure all the column types were correctly inferred

In [0]:
print(googlep.printSchema())

Looks like we need to edit some of the datatypes. Let's just update Rating, Reviews and Price as integer (float for Rating) values for now, since the Size and Installs variables will need a bit more cleaning.

In [0]:
from pyspark.sql.types import IntegerType, FloatType
df = googlep.withColumn("Rating", googlep["Rating"].cast(FloatType())) \
            .withColumn("Reviews", googlep["Reviews"].cast(IntegerType())) \
            .withColumn("Price", googlep["Price"].cast(IntegerType()))
print(df.printSchema())
df.limit(5).toPandas()

Looks like that worked! Great! Let's dig in. 

## S7.1. Create Tempview

Go ahead and create a tempview of the dataframe so we can work with it in spark sql.

In [0]:
# Create a temporary view of the dataframe
df.createOrReplaceTempView("tempview")

## S7.2. Select all apps with ratings above 4.1

Use your tempview to select all apps with ratings above 4.1

In [0]:
# Then Query the temp view
spark.sql("SELECT * FROM tempview WHERE Rating > 4.1").limit(5).toPandas()

## S7.3. Now pass your results to an object (ie create a spark dataframe)

Select just the App and Rating column where the Category is in the Comic category and the Rating is above 4.5.

In [0]:
# Or pass it to an object
sql_results = spark.sql("SELECT App,Rating FROM tempview WHERE Category = 'COMICS' AND Rating > 4.5")
sql_results.limit(5).toPandas()

## S7.4. Which category has the most cumulative reviews

Only select the one category with the most reivews. 

*Note: will require adding all the review together for each category*

In [0]:
spark.sql("SELECT Category, sum(Reviews) AS Total_Reviews FROM tempview GROUP BY Category ORDER BY Total_Reviews DESC").limit(1).toPandas()

## S7.5. Which App has the most reviews?

Display ONLY the top result

Include only the App column and the Reviews column.

In [0]:
spark.sql("SELECT App, Reviews FROM tempview ORDER BY Reviews DESC").show(1)

## S7.6. Select all apps that contain the word 'dating' anywhere in the title

*Note: we did not cover this in the lecture. You'll have to use your SQL knowledge :) Google it if you need to.*

In [0]:
spark.sql("SELECT * FROM tempview WHERE App LIKE '%dating%'").limit(5).toPandas()

## S7.7. Use SQL Transformer to display how many free apps there are in this list

In [0]:
# First we need to import SQL transformer
from pyspark.ml.feature import SQLTransformer

In [0]:
sqlTrans = SQLTransformer(
    statement="SELECT count(*) FROM __THIS__ WHERE Type = 'Free'") 
sqlTrans.transform(df).show()

## S7.8. What is the most popular Genre?

Which genre appears most often in the dataframe. Show only the top result.

In [0]:
sqlTrans = SQLTransformer(
    statement="SELECT Genres, count(*) as Total FROM __THIS__ GROUP BY Genres ORDER BY Total DESC") 
sqlTrans.transform(df).show(1)

## S7.9. Select all the apps in the 'Tools' genre that have more than 100 reviews

In [0]:
sqlTrans = SQLTransformer(
    statement="SELECT App, Reviews FROM __THIS__ WHERE Genres = 'Tools' AND Reviews > 100") 
sqlTrans.transform(df).show(10)

### That's it! Great Job!